In [1]:
import os
import numpy as np
import random
from past.builtins import xrange
import math
import operator

In [2]:
class KNearestNeighbor(object):
  """ a kNN classifier with L2 distance """

  def __init__(self):
    pass

  def train(self, X, y):
    """
    Train the classifier. For k-nearest neighbors this is just 
    memorizing the training data.
    Inputs:
    - X: A numpy array of shape (num_train, D) containing the training data
      consisting of num_train samples each of dimension D.
    - y: A numpy array of shape (N,) containing the training labels, where
         y[i] is the label for X[i].
    """
    self.X_train = X
    self.y_train = y
    
  def predict(self, X, k=1, num_loops=0):
    """
    Predict labels for test data using this classifier.
    Inputs:
    - X: A numpy array of shape (num_test, D) containing test data consisting
         of num_test samples each of dimension D.
    - k: The number of nearest neighbors that vote for the predicted labels.
    - num_loops: Determines which implementation to use to compute distances
      between training points and testing points.
    Returns:
    - y: A numpy array of shape (num_test,) containing predicted labels for the
      test data, where y[i] is the predicted label for the test point X[i].  
    """
    if num_loops == 0:
      dists = self.compute_distances_no_loops(X)

    return self.predict_labels(dists, k=k)

  

  def compute_distances_no_loops(self, X):
    """
    Compute the distance between each test point in X and each training point
    in self.X_train using no explicit loops.
    Input / Output: Same as compute_distances_two_loops
    """
    num_test = X.shape[0]
    num_train = self.X_train.shape[0]
    dists = np.zeros((num_test, num_train)) 
    

    # L2 distance vectorized.
    X_squared = np.sum(X**2,axis=1)
    Y_squared = np.sum(self.X_train**2,axis=1)
    XY = np.dot(X, self.X_train.T)

    # Expand L2 distance formula to get L2(X,Y) = sqrt((X-Y)^2) = sqrt(X^2 + Y^2 -2XY)
    dists = np.sqrt(X_squared[:,np.newaxis] + Y_squared -2*XY)
    return dists

  def predict_labels(self, dists, k=1):
    """
    Given a matrix of distances between test points and training points,
    predict a label for each test point.
    Inputs:
    - dists: A numpy array of shape (num_test, num_train) where dists[i, j]
      gives the distance betwen the ith test point and the jth training point.
    Returns:
    - y: A numpy array of shape (num_test,) containing predicted labels for the
      test data, where y[i] is the predicted label for the test point X[i].  
    """
    num_test = dists.shape[0]
    y_pred = np.zeros(num_test)
    for i in range(num_test):
      # A list of length k storing the labels of the k nearest neighbors to
      # the ith test point.
      closest_y = []

      # Select a test row.
      test_row = dists[i,:]

      # np.argsort returns indices of sorted input.
      sorted_row = np.argsort(test_row)

      # Get the k closest indices.
      closest_y = self.y_train[sorted_row[0:k]]

      # Find the most occuring index in our closest k.
      y_pred[i] = np.argmax(np.bincount(closest_y))

    return y_pred

In [3]:
def load_NMNIST(path):
    xs_train = []
    ys_train = []
    xs_test = []
    ys_test = []

    for class_index in range(0, 10):
        for (root, dirs, dat_files) in os.walk('{0}/n_Train/{1}'.format(path, str(class_index))):
            for file in dat_files:
                single_X = np.fromfile('{0}/n_Train/{1}/{2}'.format(path, str(class_index), file), dtype=np.int32)
                xs_train.append(single_X)
                ys_train.append(class_index)

        for (root, dirs, dat_files) in os.walk('{0}/n_Test/{1}'.format(path, str(class_index))):
            for file in dat_files:
                xs_test.append(np.fromfile('{0}/n_Test/{1}/{2}'.format(path, str(class_index), file), dtype=np.int32))
                ys_test.append(class_index)

    X_train = np.array(xs_train)
    y_train = np.array(ys_train)
    X_test = np.array(xs_test)
    y_test = np.array(ys_test)

    return X_train, y_train, X_test, y_test

In [4]:
data_set_path = 'C:/Users/Justin/Documents/LowPowerActionRecognition/CNN/datasets'
data = load_NMNIST(data_set_path)

#initialise data

X_train = data[0]
y_train = data[1]
X_test = data[2]
y_test = data[3]

# As a sanity check, we print out the size of the training and test data.
print('Training data shape: ', X_train.shape)
print('Training labels shape: ', y_train.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

Training data shape:  (60000, 1156)
Training labels shape:  (60000,)
Test data shape:  (10000, 1156)
Test labels shape:  (10000,)


In [5]:
# Subsample the data for more efficient code execution in this exercise
num_training = 60000
mask = list(range(num_training))
X_train = X_train[mask]
y_train = y_train[mask]

num_test = 10000
mask = list(range(num_test))
X_test = X_test[mask]
y_test = y_test[mask]

In [6]:
# flatten out all images to be one-dimensional
X_train = X_train.reshape(X_train.shape[0], 34 * 34 * 1) # Ptr_rows becomes [number of training sets] x 2312
X_test = X_test.reshape(X_test.shape[0], 34 * 34 * 1) # Pte_rows becomes [number of testing sets] x 2312
print(X_train.shape, X_test.shape)

(60000, 1156) (10000, 1156)


In [7]:
#from kNN_classifier import KNearestNeighbor

# Create a kNN classifier instance. 
# Remember that training a kNN classifier is a noop: 
# the Classifier simply remembers the data and does no further processing 
classifier = KNearestNeighbor()
classifier.train(X_train, y_train)

In [8]:
dists = classifier.compute_distances_no_loops(X_test)
print(dists.shape) #Check dists dimension
# Now implement the function predict_labels and run the code below:
# We use k = 1 (which is Nearest Neighbor).
y_test_pred = classifier.predict_labels(dists, k=1)

# Compute and print the fraction of correctly predicted examples
num_correct = np.sum(y_test_pred == y_test)
accuracy = float(num_correct) / num_test
print('Got %d / %d correct => accuracy: %f' % (num_correct, num_test, accuracy))

(10000, 60000)
Got 9258 / 10000 correct => accuracy: 0.925800


In [9]:
#k=5 test
y_test_pred = classifier.predict_labels(dists, k=5)
num_correct = np.sum(y_test_pred == y_test)
accuracy = float(num_correct) / num_test
print('Got %d / %d correct => accuracy: %f' % (num_correct, num_test, accuracy))

Got 9308 / 10000 correct => accuracy: 0.930800


In [10]:
num_folds = 5
k_choices = [1, 3, 5, 8, 10, 12, 15, 20, 50, 100]

X_train_folds = []
y_train_folds = []

# Split up the training data into folds 
# y_train_folds[i] is the label vector for the points in X_train_folds[i].     

X_train_folds = np.array(np.array_split(X_train, num_folds))
y_train_folds = np.array(np.array_split(y_train, num_folds))


# A dictionary holding the accuracies for different values of k that we find
# when running cross-validation. After running cross-validation,
# k_to_accuracies[k] should be a list of length num_folds giving the different
# accuracy values that we found when using that value of k.
k_to_accuracies = {}

# Perform k-fold cross validation to find the best value of k. For each        
# possible value of k, run the k-nearest-neighbor algorithm num_folds times,   
# where in each case you use all but one of the folds as training data and the 
# last fold as a validation set. Store the accuracies for all fold and all     
# values of k in the k_to_accuracies dictionary.                               

for k in k_choices:
    for n in xrange(num_folds):
        combinat = [x for x in xrange(num_folds) if x != n] 
        x_training_dat = np.concatenate(X_train_folds[combinat])
        y_training_dat = np.concatenate(y_train_folds[combinat])
        classifier_k = KNearestNeighbor()
        classifier_k.train(x_training_dat, y_training_dat)
        y_cross_validation_pred = classifier_k.predict_labels(X_train_folds[n], k)
        num_correct = np.sum(y_cross_validation_pred == y_train_folds[n])
        accuracy = float(num_correct) / num_test
        k_to_accuracies.setdefault(k, []).append(accuracy)

# Print out the computed accuracies
for k in sorted(k_to_accuracies):
    for accuracy in k_to_accuracies[k]:
        #print('k = %d, accuracy = '%f' % (k, accuracy))
        print('k = %d, accuracy = %f' % (k, accuracy))
    print('mean for k=%d is %f' % (k, np.mean(k_to_accuracies[k])))

k = 1, accuracy = 0.390300
k = 1, accuracy = 0.000000
k = 1, accuracy = 0.000000
k = 1, accuracy = 0.000000
k = 1, accuracy = 0.000000
mean for k=1 is 0.078060
k = 3, accuracy = 0.436500
k = 3, accuracy = 0.000000
k = 3, accuracy = 0.000000
k = 3, accuracy = 0.000000
k = 3, accuracy = 0.000000
mean for k=3 is 0.087300
k = 5, accuracy = 0.456600
k = 5, accuracy = 0.000000
k = 5, accuracy = 0.000000
k = 5, accuracy = 0.000000
k = 5, accuracy = 0.000000
mean for k=5 is 0.091320
k = 8, accuracy = 0.507700
k = 8, accuracy = 0.000000
k = 8, accuracy = 0.000000
k = 8, accuracy = 0.000000
k = 8, accuracy = 0.000000
mean for k=8 is 0.101540
k = 10, accuracy = 0.516000
k = 10, accuracy = 0.000000
k = 10, accuracy = 0.000000
k = 10, accuracy = 0.000000
k = 10, accuracy = 0.000000
mean for k=10 is 0.103200
k = 12, accuracy = 0.520400
k = 12, accuracy = 0.000000
k = 12, accuracy = 0.000000
k = 12, accuracy = 0.000000
k = 12, accuracy = 0.000000
mean for k=12 is 0.104080
k = 15, accuracy = 0.510700
